In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as opt
from sklearn.metrics import roc_auc_score,roc_curve,plot_roc_curve,auc,accuracy_score,precision_score,recall_score,f1_score
from sklearn import linear_model
import copy
from sklearn.decomposition import PCA

In [ ]:
######################### model plot #########################
infile = 'model.feature.csv'
indf = pd.read_csv(infile)
print(indf.shape)
indf.head()

In [ ]:
rcpt = ['IGH','IGK','IGL','TRA','TRB']
richness = [i+'_richness' for i in rcpt]
diversity = [i+'_diversity' for i in rcpt]
clonality = [i+'_clonality' for i in rcpt]
cdr3_pervj = [i+'_cdr3_pervj' for i in rcpt]
specific_trb_freq = ['specific_trb_freq_vjcdr3']
deg = ['SDC1', 'UCHL1', 'ADRA2A', 'IFI27', 'CA1', 'GYPB', 'C1D', 'GSTM3', 'SPIN3']
###############################
best_feature = 'IGH_diversity+TRB_diversity+TRB_clonality+UCHL1+C1D'
tag = 'immune+deg'
model_feature = best_feature.split('+')
###############################
allauc=[]; accuracy=[]; precision=[]; recall=[]; f1=[]
tprs_Health=[]; tprs_Convalescent=[]; tprs_ICU=[]; tprs_Severe=[]; tprs_Moderate=[]; tprs_Acute=[]
aucs_Health=[]; aucs_Convalescent=[]; aucs_ICU=[]; aucs_Severe=[]; aucs_Moderate=[]; aucs_Acute=[]
heatdf = pd.DataFrame(index=['Acute','Convalescent','Health'],
                      columns=['Acute','Convalescent','Health'])
plotdf = pd.DataFrame(index=['Acute','Convalescent','Health'],
                      columns=['Acute','Convalescent','Health'])
heatdf.fillna(0,inplace=True)
mean_fpr = np.linspace(0,1,100)
for cv in ['cv1','cv2','cv3']:
    traindf = indf[indf[cv]=='train']
    testdf = indf[indf[cv]=='test']
    trainx = traindf.loc[:,model_feature]
    trainy = traindf.loc[:,'type']
    testx = testdf.loc[:,model_feature]
    testy = testdf.loc[:,'type']
    lr = linear_model.LogisticRegression()
    lr.fit(trainx, trainy)
    test_prob = lr.predict_proba(testx)
    allauc.append(roc_auc_score(testy, test_prob, multi_class='ovr'))
    accuracy.append(accuracy_score(testy, lr.predict(testx)))
    precision.append(precision_score(testy, lr.predict(testx), average='macro'))
    recall.append(recall_score(testy, lr.predict(testx), average='macro'))
    f1.append(f1_score(testy, lr.predict(testx), average='macro'))
    #########################################
    probdf = pd.DataFrame(test_prob,columns=lr.classes_)
    probdf['type'] = testdf['type'].tolist()
    probdf['sample_id'] = testdf['sample_id'].tolist()
    probdf = probdf[['sample_id','type','Acute','Convalescent','Health']]
    probdf.index = probdf['sample_id'].tolist()
    probdf['predict'] = lr.predict(testx)
    ############################################
    for actual in ['Health','Convalescent','Acute']:
        thedf = probdf[probdf['type']==actual]
        n = thedf.shape[0]
        for i in thedf.index:
            row = thedf.loc[i,'type']
            col = thedf.loc[i,'predict']
            heatdf.loc[row,col] = heatdf.loc[row,col] + 1/n
    ###############################################
    df = copy.deepcopy(testdf)
    for typex in ['Health','Convalescent','Acute']:
        df['status'] = 0
        df.loc[df['type']==typex,'status'] = 1
        fpr,tpr,threshold = roc_curve(df['status'],probdf[typex])
        locals()['tprs_'+typex].append(np.interp(mean_fpr,fpr,tpr))
        locals()['tprs_'+typex][-1][0] = 0
        locals()['aucs_'+typex].append(auc(fpr,tpr))
################################
feature_file = tag+'.cv_model_best.maxcomb5.feature.txt'
with open(feature_file, 'w') as out:
    for feature in model_feature:
        out.write(feature+'\n')
################################
## ROC
allauc = []
pal = {'Acute':'orangered','Health':'wheat','Convalescent':'mediumseagreen',
      'ICU':'indigo','Severe':'violet','Moderate':'tomato'}
plt.plot([0,1],[0,1],'k--')
for typex in ['Health','Convalescent','Acute']:
    locals()['mean_tpr_'+typex] = np.mean(locals()['tprs_'+typex],axis=0)
    locals()['mean_tpr_'+typex][-1] = 1
    locals()['mean_roc_auc_'+typex] = auc(mean_fpr,locals()['mean_tpr_'+typex])
    allauc.append(locals()['mean_roc_auc_'+typex]) 
    locals()['mean_auc_'+typex] = np.mean(locals()['aucs_'+typex])
    plt.plot(mean_fpr,locals()['mean_tpr_'+typex],color=pal[typex],
             label='{} AUC={}'.format(typex,str('%.2f'%locals()['mean_roc_auc_'+typex])))
plt.xlabel('FPR', size=20)
plt.ylabel('TPR', size=20)
plt.legend(frameon=False,loc='upper left',bbox_to_anchor=(0.5,0.4))
ax = plt.axes()
plt.text(0.611,0.39,'Mean AUC = {}'.format('%.2f'%np.mean(allauc)),horizontalalignment='left',transform=ax.transAxes)
outfig = tag+'.cv_model_best.maxcomb5.roc.png'
plt.savefig(outfig, dpi=500, bbox_inches='tight')
plt.show()
################################
## heat matrix
heatdf = heatdf/3
for row in ['Acute','Convalescent','Health']:
    for col in ['Acute','Convalescent','Health']:
        if heatdf.loc[row,col] != 0:
            plotdf.loc[row,col] = str('%.2f'%heatdf.loc[row,col])
        else:
            plotdf.loc[row,col] = ' '
heatdf.index = ['Acute','Conv','Health']
heatdf.columns = ['Acute','Conv','Health']
sns.heatmap(heatdf,cmap='Blues',linewidths=1,linecolor='k',square=True,annot=plotdf.values,fmt='',vmin=0,vmax=1)
plt.yticks(fontsize=12,rotation=0)
plt.xticks(fontsize=12,rotation=0)
plt.ylabel('Actual',fontsize=18)
plt.xlabel('Predicted',fontsize=18)
ax = plt.gca()
ax.xaxis.set_label_coords(0.5, 1.15)
outfig = tag+'.cv_model_best.maxcomb5.matrix.png'
plt.savefig(outfig, dpi=500, bbox_inches='tight')
plt.show()
##############################################
## PCA
pca = PCA()
pca.fit(indf.loc[:,model_feature])
pcadf = pca.transform(indf.loc[:,model_feature])
pcadf = pd.DataFrame(pcadf)
pcadf['type'] = indf['type']
ratio = pca.explained_variance_ratio_

plt.figure(figsize=(5,5))
sns.scatterplot(x=0,y=1,hue='type',data=pcadf,
                alpha=0.85, edgecolor='k',palette=pal,s=60)
xlabel = 'PC1 ('+str('%.2f'%(ratio[0]*100))+'%)'
ylabel = 'PC2 ('+str('%.2f'%(ratio[1]*100))+'%)'
plt.xlabel(xlabel, fontsize=18)
plt.ylabel(ylabel, fontsize=18)
# plt.legend(frameon=False,loc='center left', bbox_to_anchor=(1,0.5))
ax = plt.axes()
handles,labels = ax.get_legend_handles_labels()
# ax.legend(handles=handles[1:], labels=labels[1:], title=False, frameon=False)
ax.legend(handles=[handles[3]]+[handles[2]]+[handles[1]]+handles[4:], labels=[labels[3]]+[labels[2]]+[labels[1]]+labels[4:], title=False, frameon=False,
         loc='upper left',bbox_to_anchor=(1,1))
outfig = tag+'.cv_model_best.maxcomb5.pca.png'
plt.savefig(outfig, dpi=500, bbox_inches='tight')
plt.show()


In [ ]:
# validation
rcpt = ['IGH','IGK','IGL','TRA','TRB']
richness = [i+'_richness' for i in rcpt]
diversity = [i+'_diversity' for i in rcpt]
clonality = [i+'_clonality' for i in rcpt]
cdr3_pervj = [i+'_cdr3_pervj' for i in rcpt]
specific_trb_freq = ['specific_trb_freq_vjcdr3']
deg = ['SDC1', 'UCHL1', 'ADRA2A', 'IFI27', 'CA1', 'GYPB', 'C1D', 'GSTM3', 'SPIN3']
###############################
model_feature = best_feature.split('+')
###############################
heatdf = pd.DataFrame(index=['Acute','Convalescent','Health'],
                      columns=['Acute','Convalescent','Health'])
plotdf = pd.DataFrame(index=['Acute','Convalescent','Health'],
                      columns=['Acute','Convalescent','Health'])
heatdf.fillna(0,inplace=True)
mean_fpr = np.linspace(0,1,100)
cv = 'cv1'
traindf = indf[indf[cv]!='validation']
testdf = indf[indf[cv]=='validation']
trainx = traindf.loc[:,model_feature]
trainy = traindf.loc[:,'type']
testx = testdf.loc[:,model_feature]
testy = testdf.loc[:,'type']
lr = linear_model.LogisticRegression()
lr.fit(trainx, trainy)
test_prob = lr.predict_proba(testx)
#########################################
probdf = pd.DataFrame(test_prob,columns=lr.classes_)
probdf['type'] = testdf['type'].tolist()
probdf['sample_id'] = testdf['sample_id'].tolist()
probdf = probdf[['sample_id','type','Acute','Convalescent','Health']]
probdf.index = probdf['sample_id'].tolist()
probdf['predict'] = lr.predict(testx)
############################################
for actual in ['Health','Convalescent','Acute']:
    thedf = probdf[probdf['type']==actual]
    n = thedf.shape[0]
    for i in thedf.index:
        row = thedf.loc[i,'type']
        col = thedf.loc[i,'predict']
        heatdf.loc[row,col] = heatdf.loc[row,col] + 1/n
###############################################
df = copy.deepcopy(testdf)
pal = {'Acute':'orangered','Health':'wheat','Convalescent':'mediumseagreen',
      'ICU':'indigo','Severe':'violet','Moderate':'tomato'}

plt.plot([0,1],[0,1],'k--')
for typex in ['Health','Convalescent','Acute']:
    df['status'] = 0
    df.loc[df['type']==typex,'status'] = 1
    fpr,tpr,threshold = roc_curve(df['status'],probdf[typex])
    plt.plot(fpr,tpr,color=pal[typex],
             label='{} AUC={}'.format(typex,str('%.2f'%auc(fpr,tpr))))
plt.xlabel('FPR', size=20)
plt.ylabel('TPR', size=20)
plt.legend(frameon=False,loc='upper left',bbox_to_anchor=(0.5,0.4))
ax = plt.axes()
outfig = tag+'.best.validation.cv_model.roc.png'
plt.savefig(outfig, dpi=500, bbox_inches='tight')
plt.show()
################################
## heat matrix
heatdf = heatdf
for row in ['Acute','Convalescent','Health']:
    for col in ['Acute','Convalescent','Health']:
        if heatdf.loc[row,col] != 0:
            plotdf.loc[row,col] = str('%.2f'%heatdf.loc[row,col])
        else:
            plotdf.loc[row,col] = ' '
heatdf.index = ['Acute','Conv','Health']
heatdf.columns = ['Acute','Conv','Health']
sns.heatmap(heatdf,cmap='Blues',linewidths=1,linecolor='k',square=True,annot=plotdf.values,fmt='',vmin=0,vmax=1)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.ylabel('Actual',fontsize=18)
plt.xlabel('Predicted',fontsize=18)
ax = plt.gca()
ax.xaxis.set_label_coords(0.5, 1.15)
outfig = tag+'.best.validation.cv_model.matrix.png'
plt.savefig(outfig, dpi=500, bbox_inches='tight')
plt.show()